In [ ]:
const express = require('express');
const tourController = require('./../controllers/tourController');

const router = express.Router();

router.param('id', (req,res,next,val) =>{
    console.log(`Tour id is: ${val}`);
    next();
})

router
    .route('/:id')
    .get(tourController.getTour)
    .post(tourController.updateTour);

In [1]:
//lets have a look at our route handlers / controllers

In [2]:

//Controllers aka. Router Handlers
// consider this route handler
// see how the id check takes place here.
//exports.deleteTour = (req,res) => {
//    if(req.params.id * 1 > tour.length){
//        return res.status(404).json({
//            status:"fail",
//            message:"invalid ID"
//        });
//    }
//    res.status(204).json({
//        status:'success',
//        data:null
//    });
//};

//The code in the above controller will be replaced with the following 

//tourController.js
exports.checkID = (req, res, next, val) => {
  console.log(`Tour id is: ${val}`);
  if (req.params.id * 1 > tours.length) {
    return res.status(404).json({
      status: 'fail',
      message: 'Invalid ID'
    });
  }
  next();
};



SyntaxError: Unexpected token '}'

In [ ]:
// the id check is now added to our routes/tourRoutes.js and will be executed once before our Route handler

//routes/tourRoutes.js
const express = require('express');
const tourController = require('./../controllers/tourController');

const router = express.Router();

router.param('id', tourController.checkID);

router
  .route('/')
  .get(tourController.getAllTours)
  .post(tourController.checkBody, tourController.createTour);

router
  .route('/:id')
  .get(tourController.getTour)
  .patch(tourController.updateTour)
  .delete(tourController.deleteTour);

module.exports = router;



In [ ]:
// finally our tourController.js file looks as follows:
// contoller/tourController.js
const fs = require('fs');

const tours = JSON.parse(
  fs.readFileSync(`${__dirname}/../dev-data/data/tours-simple.json`)
);


//contollers
exports.checkID = (req, res, next, val) => {
  console.log(`Tour id is: ${val}`);

  if (req.params.id * 1 > tours.length) {
    return res.status(404).json({
      status: 'fail',
      message: 'Invalid ID'
    });
  }
  next();
};

exports.checkBody = (req, res, next) => {
  if (!req.body.name || !req.body.price) {
    return res.status(400).json({
      status: 'fail',
      message: 'Missing name or price'
    });
  }
  next();
};

exports.getAllTours = (req, res) => {
  console.log(req.requestTime);

  res.status(200).json({
    status: 'success',
    requestedAt: req.requestTime,
    results: tours.length,
    data: {
      tours
    }
  });
};

exports.getTour = (req, res) => {
  console.log(req.params);
  const id = req.params.id * 1;

  const tour = tours.find(el => el.id === id);

  res.status(200).json({
    status: 'success',
    data: {
      tour
    }
  });
};

exports.createTour = (req, res) => {
  // console.log(req.body);

  const newId = tours[tours.length - 1].id + 1;
  const newTour = Object.assign({ id: newId }, req.body);

  tours.push(newTour);

  fs.writeFile(
    `${__dirname}/dev-data/data/tours-simple.json`,
    JSON.stringify(tours),
    err => {
      res.status(201).json({
        status: 'success',
        data: {
          tour: newTour
        }
      });
    }
  );
};

exports.updateTour = (req, res) => {
  res.status(200).json({
    status: 'success',
    data: {
      tour: '<Updated tour here...>'
    }
  });
};

exports.deleteTour = (req, res) => {
  res.status(204).json({
    status: 'success',
    data: null
  });
};
